In [2]:
import json
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

In [3]:
sys.path.append(os.path.abspath('../../util/model'))
from training import get_percent

In [3]:
%pwd

'c:\\Users\\Zoe\\DSA4262-ACMXZ\\data\\codes'

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [22]:
data_df11 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_1.parquet")
data_df12 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_2.parquet")
data_df13 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_3.parquet")
data_df14 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_4.parquet")
data_df15 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_5.parquet")
data_df16 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_6.parquet")
data_df17 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_7.parquet")
data_df18 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset1/dataset1_8.parquet")

In [23]:
data_df21 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_1.parquet")
data_df22 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_2.parquet")
data_df23 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_3.parquet")
data_df24 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_4.parquet")
data_df25 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_5.parquet")
data_df26 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_6.parquet")
data_df27 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/dataset2/dataset2_7.parquet")

In [24]:
raw_df =  pd.concat([data_df11, data_df12, data_df13,data_df14,data_df15,data_df16,data_df17,data_df18])
raw_df2 = pd.concat([data_df21, data_df22, data_df23,data_df24,data_df25 ,data_df26 ,data_df27 ])

In [9]:
def model_features_and_clean(df):
    '''Function to select features for modelling and clean the initial raw data. 
        Removes rows which contain NAN values in key_columns, and converts all NAN values to 0 for non_nan_cols

    :Parameters:
    ------------
        self.df: DataFrame
            DataFrame containing raw data

    :Returns:
    ---------
        self.df: DataFrame
            DataFrame containing features required for modelling
    '''
    model_features_list =  ['transcript', 'position', 'nucleotides', 'reads_count', 'gene_id',
                            'dwellingtime_-1', 'std_-1', 'mean_-1',
                            'dwellingtime_0', 'std_0', 'mean_0',
                            'dwellingtime_+1', 'std_+1', 'mean_+1','label']
    non_nan_cols =  []
    key_columns =['label']
    df = df[model_features_list]
    df[non_nan_cols] = df[non_nan_cols].fillna(0)
    df = df.dropna(subset=key_columns)
    return df

In [ ]:
def feature_eng_test(df):
    df = pd.DataFrame(df.groupby([ 'transcript', 'position', 'nucleotides', 'reads_count'], as_index=False)
                            .agg({'dwellingtime_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'std_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'mean_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'dwellingtime_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'std_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'mean_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'dwellingtime_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'std_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                    'mean_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean]}))
    df.columns = ['transcript', 'position', 'nucleotides', 'reads_count',
                        'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                        'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean',
                        'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean',
                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                        'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean',
                        'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean',
                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                        'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean',
                        'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']
    return df


In [12]:
# function to clean
def feature_eng_train(df):
    temp = pd.DataFrame(df.groupby(['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label'], as_index=False)
                           .agg({'dwellingtime_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_-1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'dwellingtime_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_0': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'dwellingtime_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'std_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean],
                                'mean_+1': [get_percent(25), get_percent(50), get_percent(75), np.mean]}))
    temp.columns = ['gene_id', 'transcript', 'position', 'nucleotides', 'reads_count', 'label',
                        'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                        'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean',
                        'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean',
                        'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                        'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean',
                        'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean',
                        'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                        'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean',
                        'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']
    return temp

In [6]:
def encoding_train(X_train_oversampled, y_train):


    one_hot_col = ['nucleotides']

    numeric_cols = ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']  
                       
    id_val = X_train_oversampled[['transcript','position']]                       

    # piping the encoding
    numeric_encoder = Pipeline([('scale', MinMaxScaler())])
    preprocessor = ColumnTransformer(transformers=[
        ("num", numeric_encoder, numeric_cols)],
        remainder='passthrough')

    # getting list of column names to map
    columns_to_map = numeric_cols + one_hot_col

    print('columns after preprocessing :', columns_to_map,  '\n')

    # applying encoding on columns in df and creating pipeline
    X_train_enc = pd.DataFrame({col: vals for vals, col in zip(
        preprocessor.fit_transform(X_train_oversampled, y_train).T, columns_to_map)})
    pipe = Pipeline(steps=[("preprocessor", preprocessor)])
    pipe = pipe.fit(X_train_enc, y_train)

    # encoding nucleotides
    for i in range(7):
        X_train_enc['position_' + str(i) + '_A'] = 0
        X_train_enc['position_' + str(i) + '_C'] = 0
        X_train_enc['position_' + str(i) + '_G'] = 0
        X_train_enc['position_' + str(i) + '_T'] = 0
        temp = X_train_enc['nucleotides'].apply(lambda x: x[i])
        X_train_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
        X_train_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
        X_train_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
        X_train_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

    # dropping nucleotides column
    X_train_enc = X_train_enc.drop(columns=['nucleotides'])

    return X_train_enc, pipe, id_val

In [111]:
def encoding_test_val(X_test, test=True):

    one_hot_col = ['nucleotides']

    numeric_cols = ['reads_count','dwelling_time_-1_25',
                    'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean',
                    'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25',
                    'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25',
                    'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean',
                    'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25',
                    'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25',
                    'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean',
                    'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25',
                    'mean_+1_50', 'mean_+1_75', 'mean_+1_mean']   
    columns_to_map = numeric_cols + one_hot_col

    id_val = X_test[['transcript','position']]

    # applying encoding on columns in df
    if test == True:  # for test
        X_test_enc = pd.DataFrame({col: vals for vals, col in zip(
            pipe.transform(X_test).T, columns_to_map)})
        # encoding nucleotides
        for i in range(7):
            X_test_enc['position_' + str(i) + '_A'] = 0
            X_test_enc['position_' + str(i) + '_C'] = 0
            X_test_enc['position_' + str(i) + '_G'] = 0
            X_test_enc['position_' + str(i) + '_T'] = 0
            temp = X_test_enc['nucleotides'].apply(lambda x: x[i])
            X_test_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
            X_test_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
            X_test_enc['position_' + str(i) + '_G'][temp == 'G'] = 1
            X_test_enc['position_' + str(i) + '_T'][temp == 'T'] = 1

        # dropping nucleotides column
        X_test_enc = X_test_enc.drop(columns=['nucleotides'])

        return X_test_enc, id_val

# To get 2 encoded test datasets

In [25]:
raw_df.head()

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1
0,ENST00000000233,244,AAGACCA,165,0.00465,2.16,127.0,0.00640,3.90,127.0,0.00797,8.75,83.7
1,ENST00000000233,244,AAGACCA,165,0.02690,4.43,106.0,0.01860,10.00,123.0,0.00863,6.20,80.0
2,ENST00000000233,244,AAGACCA,165,0.00432,3.10,108.0,0.01200,8.26,125.0,0.01590,2.89,78.7
3,ENST00000000233,244,AAGACCA,165,0.00996,4.52,123.0,0.01750,8.51,128.0,0.00498,2.63,80.0
4,ENST00000000233,244,AAGACCA,165,0.00764,2.81,124.0,0.00772,4.22,126.0,0.00474,5.84,80.9


In [26]:
raw_df2.head()

,transcript,position,nucleotides,reads_count,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1
0,AT1G01050.1,155,GAAACTA,36,0.00232,1.93,109.0,0.01260,1.97,111.0,0.00421,1.50,95.3
1,AT1G01050.1,155,GAAACTA,36,0.00896,2.27,110.0,0.00536,2.49,110.0,0.00797,2.28,96.0
2,AT1G01050.1,155,GAAACTA,36,0.00498,6.29,114.0,0.00442,2.07,111.0,0.00785,1.97,96.0
3,AT1G01050.1,155,GAAACTA,36,0.00617,5.16,106.0,0.00830,2.70,105.0,0.00199,2.82,97.4
4,AT1G01050.1,155,GAAACTA,36,0.00664,2.01,110.0,0.00495,1.89,110.0,0.01100,1.64,97.3


In [84]:
percentiles1 = feature_eng_test(raw_df)

In [105]:
percentiles1.head()

,transcript,position,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENST00000000233,244,AAGACCA,165,0.004320,0.006310,0.011000,0.008722,2.860,3.390,4.3800,3.990545,123.0,124.0,126.0,123.090909,0.005550,0.008620,0.012200,0.010218,4.3000,6.160,8.5100,6.716545,124.0,126.00,127.0,125.503030,0.003910,0.005780,0.008530,0.006773,1.8000,3.120,5.9100,3.999745,78.600,79.90,81.60,80.060606
1,ENST00000000233,261,CAAACTG,166,0.003650,0.005640,0.007970,0.006292,2.085,2.805,3.3675,2.849892,107.0,109.0,111.0,109.027711,0.004677,0.006275,0.008307,0.007084,2.5025,3.020,3.5800,3.197289,105.0,108.00,110.0,107.190964,0.003328,0.005980,0.009272,0.007168,2.0225,2.540,3.1175,2.686892,92.925,94.15,95.95,94.572289
2,ENST00000000233,316,GAAACAG,169,0.004100,0.005980,0.009300,0.007237,1.960,2.430,3.1800,2.782710,105.0,106.0,106.0,105.440237,0.003650,0.005930,0.009300,0.007050,3.0500,3.500,4.0100,3.463964,97.6,98.90,100.0,98.539053,0.004550,0.005980,0.009300,0.007141,1.5900,1.950,2.3100,2.083775,88.200,88.90,89.60,88.985799
3,ENST00000000233,332,AGAACAT,184,0.005885,0.009385,0.013425,0.010299,4.355,5.475,7.5725,6.042386,128.0,130.0,131.0,129.304348,0.005250,0.007085,0.009715,0.008195,2.1850,2.565,2.9625,2.733641,95.9,97.35,98.5,97.320652,0.003625,0.004915,0.006667,0.005806,1.6200,2.175,2.5800,2.252565,87.300,89.90,91.60,89.350000
4,ENST00000000233,368,AGGACAA,177,0.005810,0.010000,0.014600,0.011535,5.140,6.260,6.9400,6.031751,116.0,118.0,120.0,118.000000,0.008160,0.011200,0.014800,0.012108,4.6600,5.610,6.7300,5.748927,120.0,122.00,123.0,121.485876,0.006060,0.008530,0.012000,0.009484,3.0900,3.810,4.4700,3.958192,83.800,85.20,86.80,85.220904


In [85]:
percentiles2 = feature_eng_test(raw_df2)

In [88]:
X_train = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/test_data/X_train_os.parquet")
y_train = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/test_data/y_train_os.parquet")

In [106]:
X_train.head()

,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,CAAACCT,32,0.004593,0.00610,0.008528,0.006813,2.3825,2.605,3.180,2.743750,105.0,107.0,108.25,106.625000,0.003650,0.004485,0.005975,0.005411,2.8125,3.70,4.430,3.648125,94.30,95.85,98.725,97.206250,0.004148,0.006475,0.008938,0.006500,1.795,2.10,2.9175,2.430312,84.25,85.75,87.80,85.828125
1,CGGACCT,43,0.003955,0.00552,0.008790,0.006847,1.9200,2.670,3.715,3.126744,118.0,121.0,122.00,119.883721,0.004650,0.006700,0.009795,0.008437,4.4300,6.42,9.285,7.033256,118.00,121.00,124.000,120.232558,0.003605,0.005030,0.006900,0.005803,2.210,2.44,3.5150,3.198605,77.15,78.50,80.30,78.806977
2,GTAACAA,23,0.002860,0.00432,0.005935,0.004690,1.6350,2.220,3.100,2.689130,87.1,88.8,89.80,89.017391,0.004320,0.005640,0.008065,0.006497,1.8650,2.28,2.760,2.716087,91.95,93.20,94.350,93.686957,0.004440,0.005450,0.008620,0.007998,1.820,2.10,2.2650,2.232609,85.55,87.50,90.00,87.652174
3,CTGACAT,45,0.004520,0.00598,0.008300,0.006963,2.7100,3.260,3.790,3.634444,106.0,108.0,110.00,106.895556,0.003780,0.004980,0.008110,0.006140,5.8700,8.00,10.200,8.161556,118.00,121.00,122.000,119.866667,0.003870,0.005130,0.009300,0.006688,1.760,2.43,2.9900,2.522667,82.10,83.10,85.60,83.806667
4,CAAACTT,23,0.003025,0.00496,0.006180,0.005193,1.4300,2.300,2.855,2.444783,97.2,104.0,110.50,103.521739,0.004955,0.006200,0.009075,0.007418,1.9550,2.33,2.775,2.332609,106.50,108.00,110.500,108.260870,0.004485,0.006310,0.007825,0.007254,2.000,2.23,2.7400,2.432174,93.85,95.40,97.45,95.360870


In [ ]:
X_train_enc, pipe = encoding_train(X_train, y_train)

In [ ]:
X_test_enc_1, id_val_1 = encoding_test_val(percentiles1)

In [ ]:
X_test_enc_2, id_val_2 = encoding_test_val(percentiles2)

In [115]:
percentiles1.head()

,transcript,position,nucleotides,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENST00000000233,244,AAGACCA,165,0.004320,0.006310,0.011000,0.008722,2.860,3.390,4.3800,3.990545,123.0,124.0,126.0,123.090909,0.005550,0.008620,0.012200,0.010218,4.3000,6.160,8.5100,6.716545,124.0,126.00,127.0,125.503030,0.003910,0.005780,0.008530,0.006773,1.8000,3.120,5.9100,3.999745,78.600,79.90,81.60,80.060606
1,ENST00000000233,261,CAAACTG,166,0.003650,0.005640,0.007970,0.006292,2.085,2.805,3.3675,2.849892,107.0,109.0,111.0,109.027711,0.004677,0.006275,0.008307,0.007084,2.5025,3.020,3.5800,3.197289,105.0,108.00,110.0,107.190964,0.003328,0.005980,0.009272,0.007168,2.0225,2.540,3.1175,2.686892,92.925,94.15,95.95,94.572289
2,ENST00000000233,316,GAAACAG,169,0.004100,0.005980,0.009300,0.007237,1.960,2.430,3.1800,2.782710,105.0,106.0,106.0,105.440237,0.003650,0.005930,0.009300,0.007050,3.0500,3.500,4.0100,3.463964,97.6,98.90,100.0,98.539053,0.004550,0.005980,0.009300,0.007141,1.5900,1.950,2.3100,2.083775,88.200,88.90,89.60,88.985799
3,ENST00000000233,332,AGAACAT,184,0.005885,0.009385,0.013425,0.010299,4.355,5.475,7.5725,6.042386,128.0,130.0,131.0,129.304348,0.005250,0.007085,0.009715,0.008195,2.1850,2.565,2.9625,2.733641,95.9,97.35,98.5,97.320652,0.003625,0.004915,0.006667,0.005806,1.6200,2.175,2.5800,2.252565,87.300,89.90,91.60,89.350000
4,ENST00000000233,368,AGGACAA,177,0.005810,0.010000,0.014600,0.011535,5.140,6.260,6.9400,6.031751,116.0,118.0,120.0,118.000000,0.008160,0.011200,0.014800,0.012108,4.6600,5.610,6.7300,5.748927,120.0,122.00,123.0,121.485876,0.006060,0.008530,0.012000,0.009484,3.0900,3.810,4.4700,3.958192,83.800,85.20,86.80,85.220904


In [95]:
X_test_enc_1.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,165.0,0.00432,0.00631,0.011,0.008722,2.86,3.39,4.38,3.990545,123.0,124.0,126.0,123.090909,0.00555,0.00862,0.0122,0.010218,4.3,6.16,8.51,6.716545,124.0,126.0,127.0,125.50303,0.00391,0.00578,0.00853,0.006773,1.8,3.12,5.91,3.999745,78.6,79.9,81.6,80.060606,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0
1,166.0,0.00365,0.00564,0.00797,0.006292,2.085,2.805,3.3675,2.849892,107.0,109.0,111.0,109.027711,0.004677,0.006275,0.008308,0.007084,2.5025,3.02,3.58,3.197289,105.0,108.0,110.0,107.190964,0.003328,0.00598,0.009272,0.007168,2.0225,2.54,3.1175,2.686892,92.925,94.15,95.95,94.572289,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0
2,169.0,0.0041,0.00598,0.0093,0.007237,1.96,2.43,3.18,2.78271,105.0,106.0,106.0,105.440237,0.00365,0.00593,0.0093,0.00705,3.05,3.5,4.01,3.463964,97.6,98.9,100.0,98.539053,0.00455,0.00598,0.0093,0.007141,1.59,1.95,2.31,2.083775,88.2,88.9,89.6,88.985799,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
3,184.0,0.005885,0.009385,0.013425,0.010299,4.355,5.475,7.5725,6.042386,128.0,130.0,131.0,129.304348,0.00525,0.007085,0.009715,0.008195,2.185,2.565,2.9625,2.733641,95.9,97.35,98.5,97.320652,0.003625,0.004915,0.006667,0.005806,1.62,2.175,2.58,2.252565,87.3,89.9,91.6,89.35,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1
4,177.0,0.00581,0.01,0.0146,0.011535,5.14,6.26,6.94,6.031751,116.0,118.0,120.0,118.0,0.00816,0.0112,0.0148,0.012108,4.66,5.61,6.73,5.748927,120.0,122.0,123.0,121.485876,0.00606,0.00853,0.012,0.009484,3.09,3.81,4.47,3.958192,83.8,85.2,86.8,85.220904,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0


In [116]:
id_val_1.head()

,transcript,position
0,ENST00000000233,244
1,ENST00000000233,261
2,ENST00000000233,316
3,ENST00000000233,332
4,ENST00000000233,368


In [96]:
X_test_enc_2.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,36.0,0.004235,0.00531,0.00679,0.006574,1.7925,2.09,2.86,2.5575,109.0,110.0,111.0,109.888889,0.004933,0.00636,0.008773,0.007237,1.99,2.13,2.64,2.278611,108.0,110.0,110.25,108.716667,0.004285,0.005585,0.007888,0.00634,1.53,1.72,1.9825,1.804167,94.7,95.5,96.5,95.641667,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0
1,38.0,0.00498,0.005705,0.008533,0.006866,1.8725,2.465,3.1825,2.715526,83.125,84.35,85.575,84.363158,0.00432,0.00613,0.00754,0.006421,2.6525,3.03,3.3675,3.054474,92.9,93.8,94.625,93.942105,0.004055,0.005245,0.007065,0.005963,1.61,2.105,2.54,2.255,81.8,82.2,83.375,82.560526,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0
2,35.0,0.003835,0.0073,0.01028,0.008893,3.34,4.06,4.835,4.532286,113.0,121.0,124.0,119.0,0.00393,0.00797,0.0138,0.009519,5.155,7.14,9.845,7.898,124.0,125.0,127.0,124.514286,0.00465,0.00664,0.0093,0.008132,2.47,3.61,5.08,4.088857,76.6,77.8,79.3,78.12,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0
3,35.0,0.00531,0.00697,0.008465,0.007647,3.94,4.56,5.665,5.116286,117.0,118.0,120.0,117.857143,0.00365,0.00631,0.0106,0.00798,4.46,5.59,6.92,5.626829,126.5,129.0,131.0,128.314286,0.00364,0.00535,0.009065,0.007299,2.51,3.46,4.695,3.717714,79.9,80.9,84.05,81.951429,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0
4,35.0,0.004105,0.00712,0.01145,0.008467,3.86,4.33,4.935,4.520571,89.6,90.7,92.25,91.182857,0.00404,0.00564,0.008585,0.006485,6.155,7.01,8.5,7.536571,118.5,120.0,123.0,119.8,0.00465,0.00658,0.00803,0.008201,1.88,2.74,3.875,2.998,85.0,88.2,89.35,86.457143,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0


In [98]:
print(X_test_enc_1.shape)
print(X_test_enc_2.shape)

(90810, 65)
(98830, 65)


In [100]:
%pwd

'c:\\Users\\Zoe\\DSA4262-ACMXZ\\data\\codes'

In [117]:
%cd ../../data/test_data

c:\Users\Zoe\DSA4262-ACMXZ\data\test_data


In [118]:
X_test_enc_1.to_parquet("X_test_enc_1.parquet")
X_test_enc_2.to_parquet("X_test_enc_2.parquet")
id_val_1.to_parquet("df_id_val_1.parquet")
id_val_2.to_parquet("df_id_val_2.parquet")

# To get full preprocessed train dataset without oversampling

In [28]:
data_1 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/merged_data/merged_data_v2_1.parquet")
data_2 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/merged_data/merged_data_v2_1.parquet")
data_3 = pd.read_parquet("/Users/Zoe/DSA4262-ACMXZ/data/merged_data/merged_data_v2_1.parquet")
df = pd.concat([data_1, data_2, data_3])

In [29]:
df.shape

(15000000, 15)

In [30]:
df.nunique()

transcript         2127
position           5828
nucleotides         288
reads_count         925
dwellingtime_-1    1438
std_-1             1891
mean_-1             305
dwellingtime_0     1443
std_0              1892
mean_0              262
dwellingtime_+1    1359
std_+1             1879
mean_+1             378
gene_id            2084
label                 2
dtype: int64

In [10]:
clean = model_features_and_clean(df)

In [11]:
clean.shape

(15000000, 15)

In [15]:
clean.head()

,transcript,position,nucleotides,reads_count,gene_id,dwellingtime_-1,std_-1,mean_-1,dwellingtime_0,std_0,mean_0,dwellingtime_+1,std_+1,mean_+1,label
0,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00299,2.06,125.0,0.01770,10.40,122.0,0.00930,10.90,84.1,0
1,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00631,2.53,125.0,0.00844,4.67,126.0,0.01030,6.30,80.9,0
2,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00465,3.92,109.0,0.01360,12.00,124.0,0.00498,2.13,79.6,0
3,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00398,2.06,125.0,0.00830,5.01,130.0,0.00498,3.78,80.4,0
4,ENST00000000233,244,AAGACCA,185,ENSG00000004059,0.00664,2.92,120.0,0.00266,3.94,129.0,0.01300,7.15,82.2,0


In [13]:
percentiles_df = feature_eng_train(clean)

In [14]:
percentiles_df.head()

,gene_id,transcript,position,nucleotides,reads_count,label,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean
0,ENSG00000001036,ENST00000002165,1031,GAGACAG,64,0,0.003567,0.006185,0.008743,0.006725,4.0900,5.800,8.32,6.048750,116.00,119.00,122.250,119.390625,0.00529,0.009595,0.013525,0.010687,4.2425,5.385,7.6150,6.118594,122.0,124.0,125.00,123.609375,0.005875,0.009490,0.012675,0.009926,2.7925,3.830,5.1150,4.105625,80.8,81.60,82.80,81.701563
1,ENSG00000001036,ENST00000002165,1163,GAAACCC,65,0,0.005350,0.008300,0.012600,0.010138,2.2300,2.500,3.09,2.974154,104.00,105.00,106.000,105.292308,0.00465,0.006750,0.010600,0.008343,3.7700,4.430,4.9700,4.426154,97.1,98.8,102.00,99.043077,0.003650,0.005310,0.006640,0.005645,2.1300,2.480,3.0100,2.625077,81.9,82.90,84.20,83.130769
2,ENSG00000001036,ENST00000002165,1188,ATGACAC,64,0,0.003095,0.005310,0.007278,0.006283,3.6825,4.525,4.98,4.415156,93.25,95.15,97.425,95.128125,0.00424,0.006395,0.009640,0.007680,6.3700,8.095,9.9575,9.211250,119.0,122.0,123.25,121.203125,0.004940,0.005990,0.007997,0.006921,1.9250,2.320,2.9475,2.606719,87.3,88.95,90.95,88.896875
3,ENSG00000001036,ENST00000002165,1273,AGGACAG,66,0,0.005480,0.008845,0.014000,0.010680,4.8400,5.880,6.68,5.750909,115.00,116.00,117.000,115.984848,0.00832,0.010850,0.015400,0.012778,4.1600,4.995,6.1100,5.305606,119.0,121.0,122.00,120.515152,0.003650,0.004965,0.007300,0.006105,2.1700,2.820,4.1100,3.290758,82.0,83.30,84.90,83.619697
4,ENSG00000001036,ENST00000002165,1324,GAAACTA,78,0,0.007110,0.009265,0.013800,0.010030,2.8800,3.515,4.11,3.720256,105.00,106.00,107.000,105.974359,0.00398,0.005885,0.009630,0.008207,3.2000,3.760,4.3900,3.877949,102.0,104.0,106.00,104.133333,0.005510,0.007590,0.009090,0.008309,2.0300,2.495,3.0500,2.598462,92.9,93.70,94.60,93.724359


In [14]:
percentiles_df.shape

(54079, 42)

In [21]:
X_train_full = percentiles_df.drop(["label"], axis = 1)
y_train_full = percentiles_df.label.to_numpy

In [20]:
X_train_full.shape

(54079, 41)

In [22]:
y_train_full

<bound method IndexOpsMixin.to_numpy of 0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
30       0
31       0
32       0
33       0
34       0
35       0
36       0
37       0
38       0
39       0
40       0
41       0
42       0
43       0
44       0
45       0
46       0
47       0
48       0
49       0
50       0
51       0
52       0
53       0
54       0
55       0
56       0
57       0
58       0
59       0
60       0
61       0
62       0
63       0
64       0
65       0
66       0
67       0
68       0
69       0
70       0
71       0
72       0
73       0
74       0
75       0
76       0
77       0
78       0
79       0
80       0
81       0
82       0
83       0
84       0
85       0
86       0
87 

In [33]:
X_train_full_enc, __ , id_val = encoding_train(X_train_full, y_train_full)

columns after preprocessing : ['reads_count', 'dwelling_time_-1_25', 'dwelling_time_-1_50', 'dwelling_time_-1_75', 'dwelling_time_-1_mean', 'std_-1_25', 'std_-1_50', 'std_-1_75', 'std_-1_mean', 'mean_-1_25', 'mean_-1_50', 'mean_-1_75', 'mean_-1_mean', 'dwelling_time_0_25', 'dwelling_time_0_50', 'dwelling_time_0_75', 'dwelling_time_0_mean', 'std_0_25', 'std_0_50', 'std_0_75', 'std_0_mean', 'mean_0_25', 'mean_0_50', 'mean_0_75', 'mean_0_mean', 'dwelling_time_+1_25', 'dwelling_time_+1_50', 'dwelling_time_+1_75', 'dwelling_time_+1_mean', 'std_+1_25', 'std_+1_50', 'std_+1_75', 'std_+1_mean', 'mean_+1_25', 'mean_+1_50', 'mean_+1_75', 'mean_+1_mean', 'nucleotides'] 



C:\Users\Zoe\AppData\Local\Temp\ipykernel_10252\979172319.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_enc['position_' + str(i) + '_A'][temp == 'A'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_10252\979172319.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_enc['position_' + str(i) + '_C'][temp == 'C'] = 1
C:\Users\Zoe\AppData\Local\Temp\ipykernel_10252\979172319.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [29]:
X_train_full_enc.head()

,reads_count,dwelling_time_-1_25,dwelling_time_-1_50,dwelling_time_-1_75,dwelling_time_-1_mean,std_-1_25,std_-1_50,std_-1_75,std_-1_mean,mean_-1_25,mean_-1_50,mean_-1_75,mean_-1_mean,dwelling_time_0_25,dwelling_time_0_50,dwelling_time_0_75,dwelling_time_0_mean,std_0_25,std_0_50,std_0_75,std_0_mean,mean_0_25,mean_0_50,mean_0_75,mean_0_mean,dwelling_time_+1_25,dwelling_time_+1_50,dwelling_time_+1_75,dwelling_time_+1_mean,std_+1_25,std_+1_50,std_+1_75,std_+1_mean,mean_+1_25,mean_+1_50,mean_+1_75,mean_+1_mean,position_0_A,position_0_C,position_0_G,position_0_T,position_1_A,position_1_C,position_1_G,position_1_T,position_2_A,position_2_C,position_2_G,position_2_T,position_3_A,position_3_C,position_3_G,position_3_T,position_4_A,position_4_C,position_4_G,position_4_T,position_5_A,position_5_C,position_5_G,position_5_T,position_6_A,position_6_C,position_6_G,position_6_T
0,0.045314,0.10327,0.214286,0.20399,0.18683,0.348749,0.44499,0.559378,0.486364,0.657966,0.683099,0.710708,0.697847,0.291749,0.44836,0.37545,0.449242,0.330928,0.365596,0.453318,0.45268,0.738431,0.756592,0.735772,0.747428,0.472739,0.510387,0.428901,0.494663,0.217588,0.234943,0.289039,0.259058,0.392453,0.375712,0.286119,0.346844,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.046344,0.250828,0.356137,0.381919,0.410794,0.140974,0.120825,0.13104,0.166441,0.441944,0.43662,0.430052,0.445712,0.22888,0.249826,0.255067,0.295878,0.280406,0.277982,0.249228,0.288879,0.237425,0.245436,0.268293,0.238585,0.176862,0.169857,0.119097,0.146192,0.138752,0.110805,0.121544,0.118292,0.433962,0.425047,0.325779,0.399188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.045314,0.064156,0.1556,0.136416,0.157858,0.303228,0.319745,0.285831,0.316382,0.248425,0.263204,0.281952,0.263935,0.188605,0.225052,0.215557,0.252516,0.558407,0.61422,0.634066,0.752,0.678068,0.716024,0.700203,0.697587,0.348404,0.225255,0.188783,0.2501,0.114357,0.096092,0.116571,0.116547,0.637736,0.654649,0.516997,0.610372,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.047374,0.261589,0.392689,0.446494,0.446345,0.432529,0.452849,0.425061,0.455373,0.639964,0.630282,0.620035,0.636938,0.589391,0.535939,0.452619,0.586027,0.322106,0.329817,0.337191,0.373995,0.678068,0.69574,0.674797,0.683337,0.176862,0.141752,0.152977,0.183606,0.143512,0.142069,0.209071,0.181583,0.437736,0.440228,0.345609,0.417095,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.059732,0.396523,0.420858,0.437269,0.403669,0.213584,0.22053,0.214578,0.244075,0.459946,0.454225,0.447323,0.45791,0.163065,0.189463,0.215146,0.286999,0.21946,0.216514,0.204475,0.235821,0.336016,0.350913,0.349593,0.34402,0.424202,0.355601,0.244867,0.363051,0.126852,0.112184,0.124726,0.115762,0.849057,0.834915,0.620397,0.787179,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
X_train_full_enc.shape

(54079, 65)

In [34]:
id_val.head()

,transcript,position
0,ENST00000002165,1031
1,ENST00000002165,1163
2,ENST00000002165,1188
3,ENST00000002165,1273
4,ENST00000002165,1324


In [35]:
%cd ../../data/test_data

c:\Users\Zoe\DSA4262-ACMXZ\data\test_data


In [36]:
X_train_full_enc.to_parquet("X_train_full_enc.parquet")
id_val.to_parquet("df_id_val_full.parquet")